#  LangChain의 RAG 콤포넌트 - 문서 로더 (Document Loader) 

### **학습 목표**
1. LangChain의 Document Loader 개념과 BaseLoader 인터페이스를 이해한다
2. PDF, Web, JSON, CSV 등 다양한 형식의 문서를 로드할 수 있다
3. 각 로더의 매개변수를 조정하여 원하는 형태로 데이터를 추출할 수 있다
4. 메타데이터를 커스터마이징하여 문서 관리 효율성을 높일 수 있다

### **사전 준비**

**필수 라이브러리 설치:**
```bash
uv add langchain-community beautifulsoup4 pypdf jq
```

**필요한 데이터 파일:**
- data/transformer.pdf
- data/kakao_chat.json, kakao_chat.jsonl
- data/kbo_teams_2023.csv
- data/리비안_KR.txt, 테슬라_KR.txt
- articles/notionai.pdf

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

# 다양한 문서 형식 처리하기

- 역할: Document Loader는 다양한 소스에서 문서를 로드
- 구현: 
    - Document Loader는 BaseLoader 인터페이스를 통해서 구현
    - `.load()` 또는 `.lazy_load()` 메서드를 통해 동일한 방식으로 호출 
    - 대용량 데이터셋의 경우 메모리 효율을 위해 `.lazy_load()`를 사용하는 것을 권장 
- 종류:
    - PDF 파일 로더
    - 웹 페이지 로더 
    - CSV 데이터 로더
    - 디렉토리 로더
    - HTML 데이터 로더
    - JSON 데이터 로더
    - Markdown 데이터 로더
    - Microsoft Office 데이터 로더

### 1. **PDF 파일 로더**


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

In [ ]:
# 첫 번째 문서의 내용 출력
print(pdf_docs[0].page_content)

In [ ]:
# 첫 번째 문서의 메타데이터 출력
pprint(pdf_docs[0].metadata)

In [ ]:
# 비동기 로딩 (대용량 파일에 권장)
# 주의: Jupyter Notebook에서는 IPython 7.0+ 필요
# 일반 Python 스크립트에서는 async 함수 내에서 사용해야 함
async for page in pdf_loader.alazy_load():
    # 페이지별 처리
    print(page.metadata)  # 메타데이터 출력
    print(page.page_content) # 페이지 내용 출력
    print("-"*80)

### 2. **웹 문서 로더**


In [ ]:
import os
# USER_AGENT 설정 (선택사항이지만 권장)
os.environ['USER_AGENT'] = 'Mozilla/5.0 (Educational Purpose)'

from langchain_community.document_loaders import WebBaseLoader

# 기본적인 텍스트 추출
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/", 
        "https://js.langchain.com/",
        ],
    )

# 동기 로딩
web_docs = web_loader.load()

len(web_docs)

In [ ]:
web_docs[0].metadata

In [ ]:
print(web_docs[0].page_content)

In [ ]:
# 특정 HTML 요소만 파싱하고 싶을 경우 (bs4 활용)
import bs4

web_loader = WebBaseLoader(
    web_paths=["https://python.langchain.com/"],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="theme-doc-markdown markdown"),
    },
    bs_get_text_kwargs={
        "separator": " | ",    # 구분자
        "strip": True          # 공백 제거
    }
)

# 동기 로딩
web_docs = web_loader.load()

len(web_docs)

In [ ]:
print(web_docs[0].page_content)

### 3. **JSON 파일 로더**

- 설치: pip install jq 또는 uv add jq

- **jq 스키마 주요 패턴:**
    - `.messages[]`: messages 배열의 모든 요소
    - `.messages[].content`: 각 요소의 content 필드만
    - `.`: 전체 객체 선택

- **참고**: jq 문법에 대한 자세한 내용은 https://jqlang.github.io/jq/manual/ 참조

- **text_content 매개변수 설명:**
    - `text_content=True`: jq_schema로 추출한 결과를 문자열로 직접 사용 (단순 텍스트 필드 추출 시)
    - `text_content=False`: jq_schema로 추출한 결과를 JSON 객체로 처리 (복잡한 구조 추출 시)

In [ ]:
from langchain_community.document_loaders import JSONLoader 

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[].content",    # messages 배열의 content 필드만 추출
    text_content=True,                  # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

In [ ]:
json_docs[0].metadata

In [ ]:
json_docs

In [ ]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",     # messages 배열의 모든 아이템을 추출
    text_content=False,          # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

In [ ]:
# 실제 입력 데이터 확인
print(json_docs[4].page_content)  # 실제 JSON 문자열 확인

In [ ]:
# jq 스키마: sender와 content를 결합
jq_schema = '.messages[] | .sender + ": " + .content'

loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=jq_schema,
    text_content=True,
)

loader.load()

In [ ]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩되어 있음)
from langchain_core.documents import Document

decoded_json_docs = []
for doc in json_docs:

    decoded_data = json.loads(doc.page_content)
    document_obj = Document(page_content=json.dumps(decoded_data, ensure_ascii=False), metadata=doc.metadata)
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", decoded_json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", decoded_json_docs[0].page_content)


In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)


In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)


In [ ]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")

    return metadata


loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",
    content_key="content",
    metadata_func=metadata_func,
)

docs = loader.load()
pprint(docs[0].metadata)

In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    json_lines=True,
    metadata_func=metadata_func,
)

docs = loader.load()
pprint(docs[0].metadata)

---

### 실습 문제 1

다음 요구사항에 맞게 JSON 데이터를 로드하세요:
- 파일: `./data/kakao_chat.json`
- jq_schema를 사용하여 sender와 content를 결합
- 형식: "발신자: 내용"

**힌트**: jq_schema에서 문자열 결합은 `+` 연산자를 사용합니다.

In [ ]:
# 여기에 코드를 작성하세요

### 4. **CSV 파일 로더**


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

In [ ]:
## CSV 파싱 커스터마이징

csv_loader = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    csv_args={
        "delimiter": ",",               # 구분자 지정
        "quotechar": '"',               # 따옴표 문자 지정
    }
)

csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

In [ ]:
## 소스 컬럼 지정

csv_loader = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    metadata_columns=["City", "Founded"],
    content_columns=["Team", "Introduction"],
    encoding="utf-8",
    # source_column="Team"  # 이 컬럼의 값이 메타데이터의 source로 사용됨
)

csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

### 5. **Directory 로더**
- 개요: 디렉토리에서 파일들을 읽어 Document 객체로 변환
- 기능: 
    1. 파일시스템 탐색 (와일드카드 패턴 지원)
    2. 멀티스레딩을 통한 파일 I/O
    3. 인코딩 오류 등의 예외 처리

- **glob 패턴 사용법:**
    - `*.txt`: 현재 디렉토리의 모든 txt 파일
    - `**/*.txt`: 모든 하위 디렉토리의 txt 파일 (재귀 검색)
    - `data/*_2023.csv`: data 디렉토리의 _2023으로 끝나는 CSV 파일

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

# 기본 사용법
dir_loader = DirectoryLoader(
    path="./",              # 파일 경로 - 현재 디렉토리
    glob="**/*_2023.csv",     # 파일 확장자 - txt 파일만 로드
    loader_cls=CSVLoader,  # TextLoader, CSVLoader, UnstructuredFileLoader 등 지원 
    loader_kwargs={"encoding":"utf-8"}
    )
dir_docs = dir_loader.load()

print("문서의 수:", len(dir_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", dir_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", dir_docs[0].page_content)

In [ ]:
## 진행 상태바 표시
dir_loader = DirectoryLoader(
    path="./",              # 파일 경로 - 현재 디렉토리
    glob="**/*_KR.txt",     # 파일 확장자 - txt 파일만 로드
    loader_cls=TextLoader,  # TextLoader, CSVLoader, UnstructuredFileLoader 등 지원 
    show_progress=True,     # 진행 상태바 표시
    )
dir_docs = dir_loader.load()

### 실습 문제 2

다음 요구사항에 맞게 PDF 파일을 로드하고 분석하세요:

**요구사항:**
1. PDF 파일(`articles/notionai.pdf`)을 로드합니다
2. 로드된 문서의 개수를 출력합니다
3. 각 문서(페이지)의 텍스트 길이를 출력합니다

**힌트:** 
- PyMuPDFLoader 사용
- `len(doc.page_content)`로 텍스트 길이 확인
- for 루프로 각 페이지 순회

In [ ]:
# 여기에 코드를 작성하세요.